In [1]:
import configparser
import sys
import time

from botorch.sampling import SobolQMCNormalSampler
import numpy as np
import plotly.graph_objects as go
import torch

# 設定の読み込み
config = configparser.ConfigParser()
config_path = "../config.ini"
config.read(config_path)
PROJECT_DIR = config["paths"]["project_dir"]
LOG_DIR = config["paths"]["logs_dir"]
sys.path.append(PROJECT_DIR)

from src.bnn import BayesianMLPModel
from src.utils_experiment import negate_function
from src.input_trasformation import InputTransformer

In [2]:
# Function to train the model with GPU support
def fit_pytorch_model(model, num_epochs=1000, learning_rate=0.01) -> None:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        loss = -model(model.train_inputs).log_prob(model.train_targets).mean()
        print(f"Epoch {epoch+1}/{num_epochs}: loss={loss}")
        loss.backward()
        optimizer.step()

    return loss.item()

# Function to evaluate the BNN model on test data
def evaluate_bnn_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        pred_dist = model(X_test)
        test_loss = -pred_dist.log_prob(y_test).mean().item()

    print(f"Test loss: {test_loss:.6f}")
    return test_loss

# Function to fit the BNN model
def fit_bnn_model(X_train_normalized, y_train, setting_dict):
    device = setting_dict["device"]

    # Create the Bayesian MLP Model
    model_settings = setting_dict["model"]
    model = BayesianMLPModel(
        X_train_normalized,
        y_train,
        hidden_unit_size=model_settings["hidden_unit_size"],
        num_hidden_layers=model_settings["num_hidden_layers"],
    ).to(device)

    # Fit the model
    model_optim_settings = setting_dict["model_optim"]
    final_loss = fit_pytorch_model(
        model,
        num_epochs=model_optim_settings["num_epochs"],
        learning_rate=model_optim_settings["learning_rate"],
    )

    print(f"Final training loss: {final_loss:.6f}")

    return model

In [3]:
# Define the 2D objective function (negated)
def objective_function(X):
    return (X**2).sum(dim=-1)  # Simple quadratic function in 2D
objective_function = negate_function(objective_function)

# dim=2

In [40]:
# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 64,
        "num_hidden_layers": 3,
    },
    "model_optim": {
        "num_epochs": 10000,
        "learning_rate": 0.01,
    },
}

# Define the 2D search space
search_space = torch.tensor([[-10, -10], [10, 10]]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=2, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 2D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space
y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 2D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space
y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

print()
# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)

Epoch 1/10000: loss=2641.900390625
Epoch 2/10000: loss=1461.119140625
Epoch 3/10000: loss=572.344482421875
Epoch 4/10000: loss=221.57847595214844
Epoch 5/10000: loss=101.77532196044922
Epoch 6/10000: loss=46.07856369018555
Epoch 7/10000: loss=39.46412658691406
Epoch 8/10000: loss=22.021955490112305
Epoch 9/10000: loss=11.816969871520996
Epoch 10/10000: loss=7.380514621734619
Epoch 11/10000: loss=7.293600082397461
Epoch 12/10000: loss=7.4651780128479
Epoch 13/10000: loss=8.613527297973633
Epoch 14/10000: loss=6.687831401824951
Epoch 15/10000: loss=8.357426643371582
Epoch 16/10000: loss=5.324211120605469
Epoch 17/10000: loss=5.711508750915527
Epoch 18/10000: loss=5.465750217437744
Epoch 19/10000: loss=5.546260356903076
Epoch 20/10000: loss=5.6680731773376465
Epoch 21/10000: loss=5.567692279815674
Epoch 22/10000: loss=5.2548747062683105
Epoch 23/10000: loss=5.7863569259643555
Epoch 24/10000: loss=5.4536261558532715
Epoch 25/10000: loss=5.628109931945801
Epoch 26/10000: loss=5.313695907592

Epoch 127/10000: loss=5.432819366455078
Epoch 128/10000: loss=5.4741106033325195
Epoch 129/10000: loss=7.0286126136779785
Epoch 130/10000: loss=5.572167873382568
Epoch 131/10000: loss=5.392759323120117
Epoch 132/10000: loss=5.550382137298584
Epoch 133/10000: loss=5.498195171356201
Epoch 134/10000: loss=5.4972333908081055
Epoch 135/10000: loss=7.887378692626953
Epoch 136/10000: loss=5.430844306945801
Epoch 137/10000: loss=6.785219669342041
Epoch 138/10000: loss=5.383031845092773
Epoch 139/10000: loss=5.368479251861572
Epoch 140/10000: loss=5.761247634887695
Epoch 141/10000: loss=5.368116855621338
Epoch 142/10000: loss=5.348917007446289
Epoch 143/10000: loss=5.701388359069824
Epoch 144/10000: loss=5.880621433258057
Epoch 145/10000: loss=5.530273914337158
Epoch 146/10000: loss=6.485250473022461
Epoch 147/10000: loss=5.367802143096924
Epoch 148/10000: loss=5.629608154296875
Epoch 149/10000: loss=5.749397277832031
Epoch 150/10000: loss=6.686881065368652
Epoch 151/10000: loss=5.4087100028991

5.216688632965088

In [41]:
# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 256,
        "num_hidden_layers": 4,
    },
    "model_optim": {
        "num_epochs": 150,
        "learning_rate": 0.001,
    },
}

# Define the 2D search space
search_space = torch.tensor([[-10, -10], [10, 10]]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=2, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 2D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space
y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 2D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space
y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)

Epoch 1/150: loss=2061.693115234375
Epoch 2/150: loss=1622.54931640625
Epoch 3/150: loss=882.2081909179688
Epoch 4/150: loss=572.5840454101562
Epoch 5/150: loss=264.72393798828125
Epoch 6/150: loss=140.51722717285156
Epoch 7/150: loss=86.92364501953125
Epoch 8/150: loss=66.38361358642578
Epoch 9/150: loss=32.93561553955078
Epoch 10/150: loss=31.257862091064453
Epoch 11/150: loss=24.46284294128418
Epoch 12/150: loss=14.28713321685791
Epoch 13/150: loss=9.979780197143555
Epoch 14/150: loss=12.548726081848145
Epoch 15/150: loss=11.15239143371582
Epoch 16/150: loss=6.469756603240967
Epoch 17/150: loss=6.4308271408081055
Epoch 18/150: loss=5.662760257720947
Epoch 19/150: loss=5.3751983642578125
Epoch 20/150: loss=9.562280654907227
Epoch 21/150: loss=8.451726913452148
Epoch 22/150: loss=5.223247528076172
Epoch 23/150: loss=5.285620212554932
Epoch 24/150: loss=5.149983882904053
Epoch 25/150: loss=5.950787544250488
Epoch 26/150: loss=6.09153413772583
Epoch 27/150: loss=7.494956970214844
Epoch 

5.589576244354248

# dim=12

In [54]:
# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 64,
        "num_hidden_layers": 3,
    },
    "model_optim": {
        "num_epochs": 150,
        "learning_rate": 0.01,
    },
}

dim = 12  # Set the dimension to 12

# Define the search space for dim=12
search_space = torch.tensor([[-10] * dim, [10] * dim]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling for dim=12
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=dim, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 12D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space
y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling for dim=12
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 12D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space
y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)


Epoch 1/150: loss=63039.20703125
Epoch 2/150: loss=31059.55859375
Epoch 3/150: loss=11115.873046875
Epoch 4/150: loss=3827.4873046875
Epoch 5/150: loss=1661.333984375
Epoch 6/150: loss=806.9464721679688
Epoch 7/150: loss=414.7767639160156
Epoch 8/150: loss=238.25489807128906
Epoch 9/150: loss=167.5864715576172
Epoch 10/150: loss=145.65586853027344
Epoch 11/150: loss=84.333740234375
Epoch 12/150: loss=62.90330505371094
Epoch 13/150: loss=70.72589874267578
Epoch 14/150: loss=58.50360107421875
Epoch 15/150: loss=37.864967346191406
Epoch 16/150: loss=40.479618072509766
Epoch 17/150: loss=34.235107421875
Epoch 18/150: loss=25.79920196533203
Epoch 19/150: loss=33.88194274902344
Epoch 20/150: loss=19.28241729736328
Epoch 21/150: loss=17.032869338989258
Epoch 22/150: loss=23.48583221435547
Epoch 23/150: loss=21.316402435302734
Epoch 24/150: loss=12.542040824890137
Epoch 25/150: loss=9.502544403076172
Epoch 26/150: loss=10.135906219482422
Epoch 27/150: loss=14.635915756225586
Epoch 28/150: loss

11.583500862121582

In [53]:
# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 256,
        "num_hidden_layers": 4,
    },
    "model_optim": {
        "num_epochs": 150,
        "learning_rate": 0.01,
    },
}

dim = 12  # Set the dimension to 12

# Define the search space for dim=12
search_space = torch.tensor([[-10] * dim, [10] * dim]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling for dim=12
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=dim, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 12D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space
y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling for dim=12
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 12D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space
y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)


Epoch 1/150: loss=69679.0234375
Epoch 2/150: loss=282.4425354003906
Epoch 3/150: loss=21.457395553588867
Epoch 4/150: loss=7.904307842254639
Epoch 5/150: loss=8.143836975097656
Epoch 6/150: loss=7.234703063964844
Epoch 7/150: loss=8.646430015563965
Epoch 8/150: loss=7.680874347686768
Epoch 9/150: loss=10.288944244384766
Epoch 10/150: loss=8.872743606567383
Epoch 11/150: loss=10.360946655273438
Epoch 12/150: loss=9.314373970031738
Epoch 13/150: loss=8.60548210144043
Epoch 14/150: loss=8.691667556762695
Epoch 15/150: loss=8.663148880004883
Epoch 16/150: loss=10.499691009521484
Epoch 17/150: loss=8.788451194763184
Epoch 18/150: loss=8.926950454711914
Epoch 19/150: loss=8.956158638000488
Epoch 20/150: loss=9.096122741699219
Epoch 21/150: loss=9.821637153625488
Epoch 22/150: loss=9.323142051696777
Epoch 23/150: loss=9.058030128479004
Epoch 24/150: loss=9.290529251098633
Epoch 25/150: loss=9.90987491607666
Epoch 26/150: loss=9.242793083190918
Epoch 27/150: loss=9.274917602539062
Epoch 28/150

9.495650291442871

# 離散

# dim = 2

In [56]:
import torch

# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 64,
        "num_hidden_layers": 3,
    },
    "model_optim": {
        "num_epochs": 150,
        "learning_rate": 0.01,
    },
}

dim = 2  # Set the dimension to 12

# Define the search space for dim=12
search_space = torch.tensor([[-10] * dim, [10] * dim]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling for dim=12
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=dim, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 12D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space

# Discretize the training data by rounding to the nearest integer
X_train = torch.round(X_train)

y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling for dim=12
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 12D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space

# Discretize the test data by rounding to the nearest integer
X_test = torch.round(X_test)

y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)


Epoch 1/150: loss=2261.185302734375
Epoch 2/150: loss=938.9254150390625
Epoch 3/150: loss=273.81353759765625
Epoch 4/150: loss=111.59864044189453
Epoch 5/150: loss=57.26093673706055
Epoch 6/150: loss=22.102310180664062
Epoch 7/150: loss=19.35342788696289
Epoch 8/150: loss=25.527917861938477
Epoch 9/150: loss=11.698283195495605
Epoch 10/150: loss=6.8961029052734375
Epoch 11/150: loss=5.6414618492126465
Epoch 12/150: loss=7.656054496765137
Epoch 13/150: loss=7.2057905197143555
Epoch 14/150: loss=7.614027500152588
Epoch 15/150: loss=5.253983497619629
Epoch 16/150: loss=5.246617794036865
Epoch 17/150: loss=5.236344814300537
Epoch 18/150: loss=6.312527179718018
Epoch 19/150: loss=5.634456157684326
Epoch 20/150: loss=5.282569408416748
Epoch 21/150: loss=5.641075134277344
Epoch 22/150: loss=5.263918876647949
Epoch 23/150: loss=5.306118011474609
Epoch 24/150: loss=6.67186975479126
Epoch 25/150: loss=7.612152576446533
Epoch 26/150: loss=5.274707317352295
Epoch 27/150: loss=5.831804275512695
Epo

6.695545673370361

In [58]:
import torch

# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 256,
        "num_hidden_layers": 4,
    },
    "model_optim": {
        "num_epochs": 150,
        "learning_rate": 0.01,
    },
}

dim = 2  # Set the dimension to 12

# Define the search space for dim=12
search_space = torch.tensor([[-10] * dim, [10] * dim]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling for dim=12
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=dim, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 12D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space

# Discretize the training data by rounding to the nearest integer
X_train = torch.round(X_train)

y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling for dim=12
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 12D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space

# Discretize the test data by rounding to the nearest integer
X_test = torch.round(X_test)

y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)


Epoch 1/150: loss=2545.590576171875
Epoch 2/150: loss=10.945418357849121
Epoch 3/150: loss=5.749022483825684
Epoch 4/150: loss=6.134020805358887
Epoch 5/150: loss=7.545785903930664
Epoch 6/150: loss=7.263721942901611
Epoch 7/150: loss=7.432051658630371
Epoch 8/150: loss=8.128899574279785
Epoch 9/150: loss=7.664365768432617
Epoch 10/150: loss=8.43847942352295
Epoch 11/150: loss=8.492193222045898
Epoch 12/150: loss=8.159326553344727
Epoch 13/150: loss=9.888198852539062
Epoch 14/150: loss=8.250112533569336
Epoch 15/150: loss=8.324418067932129
Epoch 16/150: loss=9.88534164428711
Epoch 17/150: loss=8.4706449508667
Epoch 18/150: loss=10.22081470489502
Epoch 19/150: loss=8.859418869018555
Epoch 20/150: loss=10.720651626586914
Epoch 21/150: loss=8.903512954711914
Epoch 22/150: loss=10.192830085754395
Epoch 23/150: loss=8.680967330932617
Epoch 24/150: loss=8.785280227661133
Epoch 25/150: loss=10.873144149780273
Epoch 26/150: loss=11.129548072814941
Epoch 27/150: loss=9.827486991882324
Epoch 28/

5.609229564666748

# dim = 12

In [59]:
import torch

# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 64,
        "num_hidden_layers": 3,
    },
    "model_optim": {
        "num_epochs": 150,
        "learning_rate": 0.01,
    },
}

dim = 12  # Set the dimension to 12

# Define the search space for dim=12
search_space = torch.tensor([[-10] * dim, [10] * dim]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling for dim=12
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=dim, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 12D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space

# Discretize the training data by rounding to the nearest integer
X_train = torch.round(X_train)

y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling for dim=12
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 12D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space

# Discretize the test data by rounding to the nearest integer
X_test = torch.round(X_test)

y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)


Epoch 1/150: loss=66690.703125
Epoch 2/150: loss=32591.1171875
Epoch 3/150: loss=11530.310546875
Epoch 4/150: loss=3847.461669921875
Epoch 5/150: loss=1591.194580078125
Epoch 6/150: loss=861.3101806640625
Epoch 7/150: loss=394.6720886230469
Epoch 8/150: loss=270.94268798828125
Epoch 9/150: loss=161.41578674316406
Epoch 10/150: loss=113.25362396240234
Epoch 11/150: loss=109.22560119628906
Epoch 12/150: loss=70.39849090576172
Epoch 13/150: loss=77.04975128173828
Epoch 14/150: loss=38.62208938598633
Epoch 15/150: loss=49.17052459716797
Epoch 16/150: loss=53.754554748535156
Epoch 17/150: loss=26.449678421020508
Epoch 18/150: loss=25.478748321533203
Epoch 19/150: loss=21.490581512451172
Epoch 20/150: loss=22.862308502197266
Epoch 21/150: loss=36.902042388916016
Epoch 22/150: loss=16.837722778320312
Epoch 23/150: loss=27.77535629272461
Epoch 24/150: loss=13.547334671020508
Epoch 25/150: loss=19.839000701904297
Epoch 26/150: loss=17.255767822265625
Epoch 27/150: loss=14.367464065551758
Epoch 

5.986320972442627

In [60]:
import torch

# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 50,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 256,
        "num_hidden_layers": 4,
    },
    "model_optim": {
        "num_epochs": 150,
        "learning_rate": 0.01,
    },
}

dim = 12  # Set the dimension to 12

# Define the search space for dim=12
search_space = torch.tensor([[-10] * dim, [10] * dim]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling for dim=12
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=dim, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 12D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space

# Discretize the training data by rounding to the nearest integer
X_train = torch.round(X_train)

y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling for dim=12
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 12D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space

# Discretize the test data by rounding to the nearest integer
X_test = torch.round(X_test)

y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)


Epoch 1/150: loss=70311.9296875
Epoch 2/150: loss=446.1093444824219
Epoch 3/150: loss=26.83072853088379
Epoch 4/150: loss=11.307242393493652
Epoch 5/150: loss=8.058979988098145
Epoch 6/150: loss=7.016077518463135
Epoch 7/150: loss=8.020373344421387
Epoch 8/150: loss=7.641779899597168
Epoch 9/150: loss=9.263762474060059
Epoch 10/150: loss=8.40963077545166
Epoch 11/150: loss=8.794082641601562
Epoch 12/150: loss=8.415340423583984
Epoch 13/150: loss=9.572794914245605
Epoch 14/150: loss=8.636061668395996
Epoch 15/150: loss=9.428449630737305
Epoch 16/150: loss=8.789557456970215
Epoch 17/150: loss=12.100550651550293
Epoch 18/150: loss=10.002701759338379
Epoch 19/150: loss=9.174192428588867
Epoch 20/150: loss=8.858420372009277
Epoch 21/150: loss=9.76174545288086
Epoch 22/150: loss=10.32876205444336
Epoch 23/150: loss=9.445113182067871
Epoch 24/150: loss=12.525135040283203
Epoch 25/150: loss=9.464195251464844
Epoch 26/150: loss=9.102571487426758
Epoch 27/150: loss=11.844913482666016
Epoch 28/15

9.605788230895996

# dim = 1

In [30]:
# Define the 2D objective function (negated)
def objective_function(X):
    return (X**2).sum(dim=-1)  # Simple quadratic function in 2D
objective_function = negate_function(objective_function)


import torch

# Example settings
settings = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "initial_data_size": 300,  # Number of initial samples
    "test_data_size": 20,  # Number of test samples
    "model": {
        "hidden_unit_size": 64,
        "num_hidden_layers": 3,
    },
    "model_optim": {
        "num_epochs": 1000,
        "learning_rate": 0.001,
    },
}

dim = 1  # Set the dimension to 12

# Define the search space for dim=12
search_space = torch.tensor([[-10] * dim, [10] * dim]).to(torch.float32).to(settings["device"])
trans = InputTransformer(search_space)

# Generate initial training data using Sobol sampling for dim=12
initial_data_size = settings["initial_data_size"]
sobol_engine = torch.quasirandom.SobolEngine(dimension=dim, scramble=True)
X_train = sobol_engine.draw(initial_data_size).to(settings["device"])  # n initial samples in 12D
X_train = search_space[0] + (search_space[1] - search_space[0]) * X_train  # Scale to search space

y_train = objective_function(X_train)

# Normalize training data
X_train_normalized = trans.normalize(X_train)
y_train = y_train.to(settings["device"])

# Fit the BNN model with the generated data
model = fit_bnn_model(X_train_normalized, y_train, settings)

# Generate test data using Sobol sampling for dim=12
test_data_size = settings["test_data_size"]
X_test = sobol_engine.draw(test_data_size).to(settings["device"])  # n test samples in 12D
X_test = search_space[0] + (search_space[1] - search_space[0]) * X_test  # Scale to search space

y_test = objective_function(X_test)

# Normalize test data
X_test_normalized = trans.normalize(X_test)
y_test = y_test.to(settings["device"])

# Evaluate the model on the test data
evaluate_bnn_model(model, X_test_normalized, y_test)

Epoch 1/1000: loss=696.481689453125
Epoch 2/1000: loss=704.8427734375
Epoch 3/1000: loss=573.0589599609375
Epoch 4/1000: loss=539.0010375976562
Epoch 5/1000: loss=597.6057739257812
Epoch 6/1000: loss=491.05792236328125
Epoch 7/1000: loss=460.3424987792969
Epoch 8/1000: loss=402.50152587890625
Epoch 9/1000: loss=388.83892822265625
Epoch 10/1000: loss=308.8726806640625
Epoch 11/1000: loss=280.3505859375
Epoch 12/1000: loss=280.4157409667969
Epoch 13/1000: loss=231.912353515625
Epoch 14/1000: loss=197.68075561523438
Epoch 15/1000: loss=213.08584594726562
Epoch 16/1000: loss=187.6389617919922
Epoch 17/1000: loss=151.49217224121094
Epoch 18/1000: loss=134.0448760986328
Epoch 19/1000: loss=144.44374084472656
Epoch 20/1000: loss=105.6714096069336
Epoch 21/1000: loss=94.37051391601562
Epoch 22/1000: loss=104.0655517578125
Epoch 23/1000: loss=89.35892486572266
Epoch 24/1000: loss=91.4161376953125
Epoch 25/1000: loss=60.18272399902344
Epoch 26/1000: loss=62.25703430175781
Epoch 27/1000: loss=65.

8.784554481506348

In [31]:
X_vis = torch.linspace(-10, 10, 100).view(-1, 1).to(settings["device"])
X_vis_normalized = trans.normalize(X_vis)
y_vis = objective_function(X_vis)
y_pred_vis = model(X_vis_normalized).mean.detach().cpu().numpy()
y_pred_vis_std = model(X_vis_normalized).stddev.detach().cpu().numpy()

In [32]:
import plotly.graph_objects as go

# Assuming y_vis, y_pred_vis, and y_pred_vis_std are available as numpy arrays from your output

# Create a Plotly figure
fig = go.Figure()

# Plot the true values
fig.add_trace(go.Scatter(
    x=X_vis.cpu().numpy().flatten(), 
    y=y_vis.flatten(), 
    mode='lines+markers', 
    name='True values',
    line=dict(color='red', width=2),
    marker=dict(size=4)
))

# Plot the predicted mean
fig.add_trace(go.Scatter(
    x=X_vis.cpu().numpy().flatten(), 
    y=y_pred_vis.flatten(), 
    mode='lines', 
    name='Predicted mean',
    line=dict(color='blue', width=2)
))

# Plot the uncertainty as a shaded area
fig.add_trace(go.Scatter(
    x=X_vis.cpu().numpy().flatten(),
    y=(y_pred_vis - y_pred_vis_std).flatten(),
    mode='lines',
    name='Lower bound',
    line=dict(color='blue', width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=X_vis.cpu().numpy().flatten(),
    y=(y_pred_vis + y_pred_vis_std).flatten(),
    mode='lines',
    name='Upper bound',
    line=dict(color='blue', width=0),
    fill='tonexty',
    fillcolor='rgba(0, 0, 255, 0.2)',
    showlegend=False
))

# Update layout
fig.update_layout(
    title="True Function vs Predicted Mean with Uncertainty",
    xaxis_title="Input",
    yaxis_title="Output",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

# Show the plot
fig.show()


In [35]:
import torch
import plotly.graph_objects as go

# Assuming X_train and y_train are tensors as shown in the image
X_train_sorted, sorted_indices = torch.sort(X_train, dim=0)
y_train_sorted = y_train[sorted_indices]

# Create a Plotly figure
fig = go.Figure()

# Plot the sorted data
fig.add_trace(go.Scatter(
    x=X_train_sorted.cpu().numpy().flatten(),
    y=y_train_sorted.cpu().numpy().flatten(),
    mode='markers+lines',
    name='Sorted Data',
    line=dict(color='blue'),
    marker=dict(size=6)
))

# Update layout
fig.update_layout(
    title="Sorted X_train vs y_train",
    xaxis_title="X_train (sorted)",
    yaxis_title="y_train",
    showlegend=True
)

# Show the plot
fig.show()
